In [8]:
from langchain_community.llms import Ollama
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter


from langchain_community.chat_models import ChatOllama

In [4]:
import os

In [ ]:
#load the llm
llm = Ollama(model = "gemma:2b-instruct-q4_0" , temperature=0.7)

chat_llm = ChatOllama(model = "gemma:2b-instruct-q4_0", temperature=0.5)

loader = UnstructuredFileLoader("Data/goog-10-k-2023 (1).pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
text_chunks = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings()
knowledge_base = FAISS.from_documents(text_chunks, embeddings)
#retrival chain
qa_chain = RetrievalQA.from_chain_type(
    retriever = knowledge_base.as_retriever(),
    llm = llm
)

question = "What is the company name here?"
response = qa_chain.invoke({"query": question})
print(response['result'])

In [9]:
#load the llm
llm = Ollama(model = "gemma:2b-instruct-q4_0" , temperature=0.7)

chat_llm = ChatOllama(model = "gemma:2b-instruct-q4_0", temperature=0.5)

In [12]:
loader = UnstructuredFileLoader("Data/goog-10-k-2023 (1).pdf")

In [13]:
documents = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/azharali/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [18]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)

In [19]:
text_chunks = text_splitter.split_documents(documents)

In [20]:
embeddings = HuggingFaceEmbeddings()

/opt/anaconda3/envs/llmchatbot/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/opt/anaconda3/envs/llmchatbot/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/anaconda3/envs/llmchatbot/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If

In [21]:
knowledge_base = FAISS.from_documents(text_chunks, embeddings)

In [22]:
#retrival chain
qa_chain = RetrievalQA.from_chain_type(
    retriever = knowledge_base.as_retriever(),
    llm = llm
)

In [24]:
question = "What is this documentation?"
response = qa_chain.invoke({"query": question})


{'query': 'What is this documentation?', 'result': "This documentation is a table of contents for the company's annual report. It provides a brief overview of the topics covered in the report, including executive compensation, security ownership, and director independence."}


In [26]:
question = "What is the company name here?"
response = qa_chain.invoke({"query": question})


In [27]:
print(response['result'])

The company name is Alphabet Inc.
